In [25]:
from github import Github
from string import Template

In [26]:
g = Github("oauth")
org = g.get_organization("jay-feng-org")

# Why Terraform For Github?
- In a normal workflow if a developer wants to be added to a certain team or repo, they would submit a Jira ticket asking for access. With Terraform (infrastructure as code), a user can simply submit a Github Pull Request with their requested changes through a few lines of code, and a GoExpedi user with the necessary permissions can accept it, immediately, providing faster access.
    - Even if this may not be the biggest issue right now since permissions are lax, this change could improve security and help as the organization grows.

- Another big benefit of Terraform is that it makes setting up Github repositories quicker, and with less mistakes. To set up a repository or new team, all you need to do is copy a code block and fill out some parameters. We can set up keys, webhooks, etc. in a module so you can create a repository without worrying about all those things - there's already a good set of defaults to work with.

- We can make sweeping changes across the org by making a single change to a file (ex. implementing branch protection in all repositories)

- Helps with security as well to fulfill insurance requirements.

- The ultimate goal is that a developer can just copy something and change a couple of lines, make a Github PR, and then they get a reposotory with good security and best practices.


# Terraform Import Guide
In order to import an existing resource into the Terraform state, we must first create a resource block with a name for that resource. Luckily, this doesn't actually have to be an explicit resource block, and instead can be created using a module.

### `terraform import` command
https://www.terraform.io/docs/cli/commands/import.html  
Usage: `terraform import [options] ADDRESS ID`
- `ADDRESS` is the existing resource address of the form
    - `resource_type.resource_name`, or 
    - `module.module_name.resource_type.resource_name`, where the first "module" is simply a keyword, "module_name" is the name of the module block implementing the module, and "resource_name" is the name of the desired resource within the referenced module  
- `ID` is the ID of the specific resource you want to map to the provided address, the ID depends on the type of resource being imported



# Import an Existing Team
In order to import an existing team, we need to import the team resource and also all the team membership resources.
Order of steps:
- Use Github API to get the team IDs, since you can't use Github team names in addresses
    - get all the additional metadata as well (team name, description, privacy)
- Use Github API to get all the organization members belonging to each team
    - get the additonal metadata as well (role)
- For each team, use the team module to generate resource configurations


One issue is that this Python Github API library does is not able to distinguish members and maintainers of a certain team. However, this should not be a very large issue because
1. Terraform would be run using the Github Auth Key of an organization owner, and member vs maintainer makes no difference for owners because they have full permissions anyways
2. If these resources are being imported into Terraform, then no one should be manually doing team maintainer actions (ex. adding a member to the team) from github.com anyways
3. If we **really** need to give someone the maintainer role, then we can still change that in Terraform by manually creating a github_team_membership resource block

In [27]:
def generate_team_config(team_id):
    team = org.get_team(team_id)
    team_name = team.name
    team_description = team.description
    team_privacy = team.privacy
    
    module_name = "make_" + team_name
    member_usernames = str([member.login for member in team.get_members()]).replace("\'", "\"")
    t = Template('module "$module_name" {\n  source = "./modules/team"\n\n  token = var.token\n  name = "$team_name"\n  privacy = "$team_privacy"\n  description = "$team_description"\n\n  usernames = $member_usernames\n}')
    return t.substitute(module_name = module_name, team_name = team.name, team_description = team_description, team_privacy = team_privacy, member_usernames = member_usernames)

In [28]:
print(generate_team_config(4955859))

module "make_team_2_manual" {
  source = "./modules/team"

  token = var.token
  name = "team_2_manual"
  privacy = "secret"
  description = "terraform import test team"

  usernames = ["jay-feng", "jay-feng-ge"]
}


### Directions
1. copy the generated team config block into a .tf file (teams.tf or imported_teams.tf)
2. run `terraform init` to make sure the resource names exist in the terraform state
3. run `terraform import module.make_team_2.github_team.some_team 4955859`

No need to import the team members, Terraform will do that automatically. Since a team member resource block would only have the username and role of a member, there is no need to go through the trouble to explicitly create a resource block for that.

In [29]:
# # probably not needed?

# def generate_import_script(team):
#     import_team_template = Template("terraform import module.team.github_team.some_team $team_id")
#     import_team = import_team_template.substitute(team_id = team.id)
    
#     team_members = list(team.get_members())
#     import_user_template = Template("terraform import module.team.github_team_membership.team_member[$i] $team_id:$username_i")
#     sep = "\n"
#     print(sep.join([import_user_template.substitute(team_id = team.id, i = i, username_i = team.get_members()[i].login)
#                    for i in range(len(team_members))]))
    
    
# print(generate_import_script(team))

# Import an Existing Organization Member


In [30]:
def generate_org_member_config(username):
    module_name = "add_" + username
    member = [member for member in org.get_members() if member.login ==  username][0]
    role = member.get_organization_membership(org).role
    
    t = Template('module "$module_name" {\n  source = "./modules/organization_member"\n\n  token = var.token\n\n  username = "$username"\n  role = "$role"\n}')
    return t.substitute(module_name = module_name, username = username, role = role)

In [31]:
print(generate_org_member_config("vinayakgajjewar"))

module "add_vinayakgajjewar" {
  source = "./modules/organization_member"

  token = var.token

  username = "vinayakgajjewar"
  role = "member"
}


### Directions
1. copy the generated organization membership config block into a .tf file (teams.tf or imported_teams.tf)
2. run `terraform init` to make sure the resource names exist in the terraform state
3. run `terraform import module.add_jay-feng-ge.github_membership.membership_for_some_user jay-feng-org:jay-feng-ge`

Normal workflow:
- dev creates jira ticket, say "i want this repo with these users having access"
    - most users won't have access to do this themselves
    - ***quicker, less mistakes***
    - we can set up a set of keys, webhooks, etc. in a module so they don't have to specify it
    - most people don't really have permissions - they can create a repo through github.com, set up permissions and webhooks separately 
    - doesn't fix self servicing (they can do they themselves alr), gives them a good set of defaults
        - easier to set up branch protection etc.
    - helps with security as well
        - fulfill insurance requirements
    - GOAL: developer can just copy something and change a couple of lines, make a github PR and they get a repo with good security and best practices

Other stuff to potentially try:
- https://www.terraform.io/docs/language/modules/testing-experiment.html